<a href="https://colab.research.google.com/github/Cseudave/automatic_tops/blob/main/01_2_Web_Scraping_anidb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping

## Anidb

Anidb es una página que almacena información de casi todos los animes. Incluso tiene una sección de etiquetas ponderadas para cada serie. Por lo que será necesario BeautifulSoup4

In [ ]:
!pip install BeautifulSoup4
!pip install cfscrape
!pip install cloudscraper

Cloudscraper será necesario para lidiar con protecciones de algunos sitios

In [ ]:
# Importamos librerías
from bs4 import BeautifulSoup
import cloudscraper
import requests
from cfscrape import create_scraper
import urllib.request

import random
import time 

import pandas as pd

In [ ]:
# Obtenemos los datos del html de algún sitio usando BeautifulSoup
def ingrediente(scraper, url):
  response = scraper.get(url)
  # Usar BS4 para procesar el contenido 
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

In [ ]:
# Configuramos un scraper para usarlo en chrome
scraper = cloudscraper.create_scraper(delay=10, browser='chrome') 

In [ ]:
# Se construyen las url para las primeras dos páginas por cada temporada del 2022
links = []
year = 2022
for season in seasons:
  for page in pages:
    links.append('https://anidb.net/anime/?h=1&noalias=1&orderby.name=1.1&orderby.ucnt=0.2&'+ str(page)+\
              '&season.month='+ str(season) + '&season.year=' + str(year) + '&view=list')

In [ ]:
# De cada página obtendremos los nombres del listado para obtener los urls por cada anime 
def anidb(db, link):
  sopa = ingrediente(scraper, link)
  titles = sopa.find_all('td', class_='name main anime')
  hrefs = sopa.find_all('td', class_='name main anime')
  for title, href in zip(titles, hrefs):
    db[title.find('a').text] = 'https://anidb.net/' + href.find('a')['href']
  return db

In [ ]:
for link in links:
  db = anidb(db, link)
  # Se utiliza una pausa de tiempo aleatorio para no ser bloqueados
  time.sleep(random.randint(2, 7))

In [ ]:
# Se guardan los datos en una DataFrame
df = pd.DataFrame([[key, db[key]] for key in  db.keys()], columns=['anime', 'link'])
# Y se guarda la lista de links y nombres de animes
df.to_csv('anidb_links22.csv')

In [ ]:
# Se busca en la sopa los datos relevantes
# En caso de no existir ese dato especifico se guarda uno alternativo
def new_dic(sopa):
  try:
    table = sopa.find_all('table')[0].find_all('td')
  except:
    print("Bloqueo")
    return 'stop'
  
  starts = sopa.find_all('span', class_='weight')
  tagnames = sopa.find_all('span', class_='tagname')
  nd = {}

  nd['name'] = table[0].find('span', itemprop='name').text
  nd['img'] = img = sopa.find('img', itemprop='image')['src']
  try:
    nd['name_en'] = table[1].find('label', itemprop='alternateName').text
  except:
    nd['name_en'] = table[0].find('span', itemprop='name').text
  try:
    nd['name_jp'] = table[2].find('label', itemprop='alternateName').text
  except: 
    nd['name_jp'] = table[0].find('span', itemprop='name').text
  nd['type'] = table[3].text.split(',')[0]
  try:
    nd['episodes'] = table[3].text.split(',')[1]
  except:
    nd['episodes'] = 1
  nd['start'] = table[4].text.split(' until ')[0]
  try:
    nd['end'] = table[4].text.split(' until ')[1]
  except:
    nd['end'] = table[4].text.split(' until ')[0]
  nd['season'] = table[5].text.split(' ')[0]
  try:
    nd['year'] = table[5].text.split(' ')[1]
  except:
    nd['year'] =  None
  genre = []
  for line in table[6].find_all('span', itemprop='genre'):
    genre.append(line.text)
  nd['genre'] = genre
  link_ex = []
  for line in table[7].find_all('a'):
    link_ex.append(line['href'])
  nd['link_ex'] = link_ex
  nd['rating'] = table[8].text.split(' ')[0]
  try:
    nd['nrating'] = table[8].text.split(' ')[1]
  except:
    nd['nrating'] = None
  nd['average'] = table[9].text.split(' ')[0]
  try: 
    nd['naverage'] = table[9].text.split(' ')[1]
  except:
    nd['naverage'] = None
  try:
    nd['rrating'] = table[10].text.split(' ')[0]
  except:
    nd['rrating'] = None
  try:
    nd['nrrating'] = table[10].text.split(' ')[1]
  except:
    nd['nrrating'] = None
  star_dic = {}
  for i in range(len(starts) - 2):
    star_dic[tagnames[i + len(nd['genre'])].text] = starts[i].text.replace('\n', '')
  nd['tags'] = star_dic
  return nd

In [ ]:
# Importamos librerías que nos permitirán ver el avance
from ipywidgets import IntProgress
from IPython.display import display

In [ ]:
# Cargamos los links
df = pd.read_csv('anidb_links22.csv')
urls = df['link']

In [ ]:
# Cargamos db22.csv para repetir el scrap
# Porque en ocaciones se bloquea nuestra IP
# Inicialmente no existe este archivo
try:
  df = pd.read_csv('db22.csv')
  data = df.to_dict(orient='records')
except:
  data = {}

In [ ]:
progress = IntProgress()
display(progress)
scraper = cloudscraper.create_scraper(delay=10, browser='chrome') 

# Obtenemos los datos deseados de cada url faltante
# Comenzando desde donde fuimos bloqueados
for i in range(len(data) - 1, len(urls)):
  url = urls[i]
  sopa = ingrediente(scraper, url)
  p = new_dic(sopa)
  if p =='stop':
    break
  prueba.append(p)
  time.sleep(random.uniform(0, 1))
  progress.value = i

In [ ]:
# Creamos el dataframe para guardarlo
df_db = pd.DataFrame.from_dict(prueba)
df_db.to_csv('db22.csv', index=False)
df_db.to_excel('db22.xlsx', index=False)